In [1]:
import pandas as pd
from pandas import read_csv, read_json

In [3]:
raw_csv = read_csv('../data/raw/reunion_segments.csv')
df = pd.DataFrame(raw_csv)
raw_parquet = pd.read_parquet('../data/raw/reunion_segments.parquet')
df_parquet = pd.DataFrame(raw_parquet)

In [8]:
df['gradient'] = (df['elevation_gain'] / df['distance']) * 100  # %
df['avg_speed'] = df['distance'] / df['best_time']  # m/s
df['avg_speed_kmh'] = df['avg_speed'] * 3.6  # km/h


In [ ]:
df_ride = df[df['activity_type'] == 'Ride'].copy()
df_run = df[df['activity_type'] == 'Run'].copy()

### Ride

In [ ]:
def uphill_factor(gradient, distance):
    if 10 < gradient <= 13 and distance > 15:
        return 4
    elif 8 < gradient <= 10 and distance > 30:
        return 4.5
    elif 5 < gradient <= 8 and distance > 60:
        return 5
    elif 3 < gradient <= 5 and distance > 120:
        return 6
    else:
        return 7

In [ ]:
def slope_factor(gradient, distance):
    if gradient <-30:
        return 1.5
    elif -30< gradient <0:
        return 1 + 2*0.7/13*gradient +0.7/13**2*gradient**2
    elif 0<= gradient <20:
        return 1 + (gradient/uphill_factor(gradient,distance))**2
    else:
        return 10

In [6]:
def expected_road_speed(gradient, distance):
    base_speed = 40/3.6  # m/s on flat road
    return base_speed / slope_factor(gradient, distance)

In [9]:
df['expected_road_speed'] = df.apply(lambda row: expected_road_speed(row['gradient'], row['distance']), axis=1)

# Speed ratio: actual / expected
# < 1 means slower than road = rough terrain
df['speed_ratio'] = df['avg_speed_kmh'] / df['expected_road_speed']

# === Consistency ratio ===
# Technical terrain = more variance in times
df['consistency_ratio'] = df['best_time'] / df['average_top_10_time']
# Closer to 1 = consistent (road), lower = high variance (technical)

### Run